In [259]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim

import json
import requests

# Web Scraping

In [260]:
from bs4 import BeautifulSoup

wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

response = requests.get(wiki_url)

soup = BeautifulSoup(response.text, 'lxml')

table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df['Neighborhood'].replace(' / ', ', ', inplace=True, regex=True) 
df = df.reset_index(drop=True)

# df[ df['Neighborhood'] == '' ] # There are no case with blank Neighborhood

df.head()
#df = df.astype({'Postal code':'String','Borough':'String' ,'Neighborhood':'String'})

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [261]:
df.shape

(103, 3)

In [262]:
df.groupby('Postal code')
df.shape

(103, 3)

# Get the latitude and the longitude coordinates

In [263]:
geolocation_file = 'Geospatial_Coordinates.csv'
gdf = pd.read_csv('https://cocl.us/Geospatial_data')
gdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Add Latitude and Longitude

In [264]:
df['Latitude'] = None
df['Longitude'] = None
df = df.astype({'Postal code':'str','Borough':'str' ,'Neighborhood':'str', 'Latitude':'float64','Longitude':'float64'})
df.dtypes

Postal code      object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [265]:
gdf = gdf.astype({'Postal Code':'str','Latitude':'float64','Longitude':'float64'})
gdf.rename(columns={'Postal Code': 'Postal code'},inplace = True)
df.dtypes

Postal code      object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [266]:
lat = list()
long = list()
for i, row in df.iterrows():
    key = df.iloc[i]['Postal code']
    for j, jrow in gdf.iterrows():
        if key == gdf.iloc[j]['Postal code']:
            lat.append( gdf.iloc[j]['Latitude'] )
            long.append( gdf.iloc[j]['Longitude'])


df['Latitude'] = lat
df['Longitude'] = long
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
